In [ ]:
import os; os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [ ]:
import tensorflow as tf
import numpy as np
import cv2
import numpy as np
from pathlib import Path
from PIL import Image
from glob import glob
from utils import *

In [ ]:
def sample_metric(img1, img2):
    return rmse(img1, img2)